# RSA

## Setup

In [1]:
from glob import glob
import numpy as np
import pandas as pd
from pathlib import Path
from src import stats, plotting, temporal, decoding, rsa

pd.set_option('display.max_rows', None)

In [2]:
subj = 'pseudo010'
process = 'EEG_RSA'
eeg_metric = 'decoding'
fmri_metric = 'decoding'
regress_gaze = False

run_splithalf_reliability = True
run_fmri_rsa = True
run_feature_decoding = True
run_feature_rsa = True

top_path = '/Users/emcmaho7/Dropbox/projects/SI_EEG/SIEEG_analysis'
data_dir = f'{top_path}/data'
figure_dir = f'{top_path}/reports/figures/{process}/{subj}'

Path(figure_dir).mkdir(exist_ok=True, parents=True)
Path(f'{data_dir}/interim/{process}').mkdir(exist_ok=True, parents=True)

In [3]:
rois = ['EVC', 'MT', 'EBA',
        'LOC', 'FFA', 'PPA',
        'pSTS', 'face-pSTS', 'aSTS']
features = ['alexnet', 'moten', 'indoor',
                 'expanse', 'object_directedness', 'agent_distance',
                 'facingness', 'joint_action', 'communication', 
                 'valence', 'arousal']
annotated_features = ['indoor', 'expanse', 'object_directedness', 
                 'agent_distance', 'facingness', 'joint_action', 
                 'communication', 'valence', 'arousal']

In [4]:
df = pd.read_csv(f'{data_dir}/interim/PreprocessData/{subj}_reg-gaze-{regress_gaze}.csv.gz')
all_cols = set(df.columns.to_list())
other_cols = set(['trial', 'time', 'offset', 'offset_eyetrack_x', 'video_name',
              'gaze_x', 'gaze_y', 'pupil_size', 'target_x', 'target_y',
               'target_distance', 'offset_eyetrack_y', 'repetition', 'even', 'session'])
channels = list(all_cols - other_cols)
feature_df = pd.read_csv(f'{data_dir}/interim/FeatureRDMs/feature_annotations.csv')

## Decoding

In [ ]:
if run_feature_decoding: 
    videos = np.array(feature_df['video_name'])
    df_avg = df.groupby(['time', 'video_name']).mean(numeric_only=True).reset_index()
    cols_to_drop = set(df.columns.to_list()) - set(['time', 'video_name'] + channels)
    df_avg.drop(columns=cols_to_drop, inplace=True)
    df_avg.sort_values(['time', 'video_name'], inplace=True)
    df_avg = temporal.smoothing(df_avg, 'video_name')

    results = decoding.eeg_feature_decoding(df_avg, feature_df,
                                            annotated_features, channels)
    results.to_csv(f'{data_dir}/interim/{process}/{subj}_feature_EEG_reg-gaze-{regress_gaze}_decoding.csv', index=False)

    out_file = f'{figure_dir}/{subj}_feature_EEG_reg-gaze-{regress_gaze}_decoding.png'
    plotting.plot_eeg_feature_decoding(out_file, results, annotated_features)

## Gaze decoding

In [5]:
if 'gaze_x' in df.columns:
    avg_df = df.groupby(['video_name', 'time']).mean().reset_index()
    gaze_df = avg_df.loc[(avg_df.time > 0) & (avg_df.time <= .5)]
    X = np.hstack([gaze_df.pivot(index='video_name', columns='time', values='gaze_x').to_numpy(),
                gaze_df.pivot(index='video_name', columns='time', values='gaze_y').to_numpy()])

    results = decoding.gaze_feature_decoding(X, feature_df, annotated_features, gaze_df.video_name.to_list())
    print(results)
    results['subj'] = subj
    results.to_csv(f'{data_dir}/interim/{process}/{subj}_feature_gaze_decoding.csv', index=False)